In [16]:
from tempfile import tempdir
import SimpleITK as sitk
import numpy as np
import numpy.ma as ma
from matplotlib import pyplot
import matplotlib.pyplot as plt
import os
import sys
#import seaborn as sns
import pandas as pd

In [17]:
patID = "1088"
patID_l = "0001088"

url_20f = 'D:/data/prostateMR_radiomics/nifti/20fractions/'
url_20f_new = 'D:/data/prostateMR_radiomics/nifti/20fractions_new/'
url_SABR = 'D:/data/prostateMR_radiomics/nifti/SABR/'
url_SABR_new = 'D:/data/prostateMR_radiomics/nifti/SABR_new/'

url = url_SABR + patID_l 
scanDir = os.listdir(url)

scan_info = pd.read_csv('D:/data/Aaron/ProstateMRL/Data/MRLPacks/ScanInfo/' + patID + '.csv')
scan_info = scan_info.dropna()

output = 'D:/data/Aaron/ProstateMRL/Data/Extraction/MRLPacks/MeanValues/' 

print("Patient Directory: " + url)
print("Output Directory: " + output)

#print(scan_info.head())


Patient Directory: D:/data/prostateMR_radiomics/nifti/SABR/0001088
Output Directory: D:/data/Aaron/ProstateMRL/Data/Extraction/MRLPacks/MeanValues/


In [18]:
def MaskedImage(image_url, mask_url):
    image = sitk.ReadImage(image_url)
    image_array = sitk.GetArrayFromImage(image)
    
    mask = sitk.ReadImage(mask_url)
    mask_array = sitk.GetArrayFromImage(mask) 
    if "glute" in mask_url:
        mask_array = mask_array / 13
    
    if image_array.shape != mask_array.shape:
       # print(image_url, mask_url)
       # print(image_array.shape, mask_array.shape)
        mean = np.nan
        std = np.nan
    
    else:
        masked_image = ma.masked_array(image_array, mask=np.logical_not(mask_array), keep_mask=True, hard_mask=True)
        mean = np.mean(masked_image.flatten())
        std = np.std(masked_image.flatten())


    return mean 
    



In [19]:
t1 = "130000"
t1 = pd.to_datetime(("20120303" +t1))
print(t1)

2012-03-03 13:00:00


In [20]:
results_df = pd.DataFrame(columns=["PatID", "MRCont", "Fraction","ScanDate", "ScanTime", "Scan", "Region", "MeanSignal", "TimeDiff","SignalDiff"])
scanValues = {"PatID":[], "MRCont":[], "Fraction":[], "ScanDate":[], "ScanTime":[], "Scan":[], "Region": [], "MeanSignal":[], "TimeDiff":[], "SignalDiff":[]}

In [25]:
fractions = scan_info.Fraction.unique()
MRs = scan_info.MRContour.unique()
#print(fractions)
for i in fractions:
    #print(i)
    MR = MRs[i].replace(' ', '')
    fraction = fractions[i]

    temp_df = scan_info.loc[scan_info['Fraction'] == fractions[i]]
    #temp_df = scan_info.loc[scan_info['Fraction'] == fractions[i]]
    print('Pat: ' + patID + ' Fraction: ' + str(fraction) + ' MR: ' + MR)

    scanValues["PatID"] = patID
    scanValues["MRCont"] = MR
    scanValues["Fraction"] = fraction
    
    scanValues["ScanDate"] = temp_df.ContourDate.unique()[0]
    scantimes = temp_df.ScanTime.unique()

    scanValues["Observer"] = "RP"
    scanValues["Region"] = "Prostate" # change when add in more

    
    numberofscans = -1
    for j in os.scandir(url + '/' + MR):
        #print(j.name)
        file = j.name
        glute_url = url + '/' + MR + '/' + patID_l + '_' + MR + '_glute.nii'
        pros_url = url + '/' + MR + '/' + patID_l + '_' + MR + '_shrunk_pros.nii'
        psoas_url = url + '/' + MR + '/' + patID_l + '_' + MR + '_psoas.nii'

        mask_urls = [pros_url, glute_url, psoas_url]

        s = -1
        if "reg" in file:
            s = s + 1
            scan = file.split("_")
            scan = scan[4]
            scan = int(scan[0:1])
            scanValues["Scan"] = scan

            scanValues["ScanTime"] = scantimes[numberofscans]

            image_url = url + '/' + MR + '/' + file
            for k in range(len(mask_urls)):
                mean = MaskedImage(image_url, mask_urls[k])
                scanValues["Mean"] = mean
                if k == 0:
                    print("Prostate: " + mask_urls[k])
                    scanValues["Region"] = "Prostate"
                elif k == 1:
                    print("Glute")
                    scanValues["Region"] = "Glute"
                elif k == 2:
                    scanValues["Region"] = "Psoas"

                if scan == 1:
                    firstmean = mean
                    scanValues["SignalDiff"] = 0
                else:
                    scanValues["SignalDiff"] = mean - firstmean
                #print(file)
                #print(" Pros: " + str(pros_image))
                results_df = results_df.append(scanValues, ignore_index=True)

                 



Pat: 1088 Fraction: 0 MR: MR6
Prostate: D:/data/prostateMR_radiomics/nifti/SABR/0001088/MR6/0001088_MR6_shrunk_pros.nii
Glute
Prostate: D:/data/prostateMR_radiomics/nifti/SABR/0001088/MR6/0001088_MR6_shrunk_pros.nii
Glute


KeyboardInterrupt: 

In [40]:
df = pd.read_csv('D:/data/Aaron/ProstateMRL/Data/MRLPacks/MeanValues/0001088.csv')



0 0
0
0 0
0
0 0
0
0 47
47
0 47
47
0 47
47
0 53
53
0 53
53
0 53
53
1 0
60
1 0
60
1 0
60
0 0
0
0 0
0
0 0
0
0 12
12
0 12
12
0 12
12
0 31
31
0 31
31
0 31
31
0 41
41
0 41
41
0 41
41
0 0
0
0 0
0
0 0
0
0 32
32
0 32
32
0 32
32
0 51
51
0 51
51
0 51
51
1 2
62
1 2
62
1 2
62
0 0
0
0 0
0
0 0
0
0 18
18
0 18
18
0 18
18
0 37
37
0 37
37
0 37
37
0 55
55
0 55
55
0 55
55
1 14
74
1 14
74
1 14
74
1 26
86
1 26
86
1 26
86
0 0
0
0 0
0
0 0
0
0 19
19
0 19
19
0 19
19
0 39
39
0 39
39
0 39
39
0 49
49
0 49
49
0 49
49
1 4
64
1 4
64
1 4
64
1 13
73
1 13
73
1 13
73
